In [1]:
import numpy as np

from air_hockey_challenge.framework import AirHockeyChallengeWrapper, ChallengeCore
from air_hockey_challenge.utils.tournament_agent_wrapper import TrainingTournamentAgentWrapper
from baseline.baseline_agent.baseline_agent import BaselineAgent

from air_hockey_agent.agent_builder import build_agent
from air_hockey_agent.raw_policy_agent import RawPolicyAgent

from mushroom_rl.utils.dataset import compute_J

In [2]:
def reward(base_env, state, action, next_state, absorbing):
    puck_pos, puck_vel = base_env.get_puck(next_state)
    q_pos, q_vel = base_env.get_joints(next_state, agent=1)
    ee_pos, ee_vel = base_env.get_ee(robot=1)

    reward = 0

    reward = ee_pos[0]
    # reward = -np.linalg.norm(action)

    # Puck is moving towards the opponent
    # reward += puck_vel[0]

    # End effector is on table
    # reward -= max(0, -1000 * (ee_pos[0] + base_env.env_info['table']['length'] / 2))
    # reward -= max(0, 1000 * (np.abs(ee_pos[1]) - base_env.env_info['table']['width'] / 2))
    # reward -= max(0, 1000 * np.abs(ee_pos[2] - base_env.env_info['robot']['ee_desired_height']))

    # if absorbing:
    #     # Puck in Goal
    #     if (np.abs(puck_pos[1]) - base_env.env_info['table']['goal_width'] / 2) <= 0:
    #         # Score for home
    #         if puck_pos[0] > base_env.env_info['table']['length'] / 2:
    #             reward += 10000
    #         # Score for opponent
    #         elif puck_pos[0] < -base_env.env_info['table']['length'] / 2:
    #             reward -= 10000

    return reward, 0

In [3]:
mdp = AirHockeyChallengeWrapper('tournament', custom_reward_function=reward, interpolation_order=(3, 3))

In [4]:
# agent_1 = build_agent(mdp.env_info)
agent_1 = RawPolicyAgent.load_agent('test.msh', mdp.env_info)

In [5]:
agent = TrainingTournamentAgentWrapper(
    mdp.env_info,
    agent_1=agent_1,
    agent_2=BaselineAgent(mdp.env_info, 2),
)

In [6]:
core = ChallengeCore(agent, mdp, is_tournament=True, init_state=mdp.base_env.init_state, time_limit=1)

In [11]:
initial_replay_size = 5000
n_steps = 4000
n_steps_test = 1000
n_epochs = 20

In [13]:
# core.learn(n_steps=initial_replay_size, n_steps_per_fit=initial_replay_size)
for epoch in range(n_epochs):
    print(f'{epoch=}')
    print('Training')
    core.learn(n_steps=n_steps, n_steps_per_fit=1)
    print('Validation')
    dataset = core.evaluate(n_steps=n_steps_test)
    J = sum(compute_J(agent.get_dataset_1(dataset)))
    print(f'{J=}')

epoch=0
Training


Validation


J=-1456.8019687449719
epoch=1
Training


Validation


J=-1051.4168627236681
epoch=2
Training


...

J=-1116.6369445785192
epoch=18
Training


Validation


J=-1159.5293922775684
epoch=19
Training


Validation


J=-1252.9885134014578


In [14]:
agent_1.save('test.msh', full_save=False)

In [7]:
dataset = core.evaluate(n_steps=1000, render=True)